In [1]:
# import libraries

import cv2
import mediapipe as mp
import csv
import numpy as np
import pandas as pd
import os

In [2]:
# instantiate mp modules

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [28]:
# load and annotate img (need to do it on a single image to create the csv)
with mp_holistic.Holistic(
    static_image_mode=True) as holistic:
    
    image = cv2.imread("../../../distracted_driver_dataset/imgs/train/c0_safe/img_34.jpg")
    image_height, image_width, _ = image.shape
    
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Draw pose, left and right hands, and face landmarks on the image.
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#     cv2.imshow('img', annotated_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [29]:
num_coords = len(results.pose_landmarks.landmark)\
            +len(results.face_landmarks.landmark)\
            +len(results.right_hand_landmarks.landmark)\
            +len(results.left_hand_landmarks.landmark)
num_coords
print(len(results.pose_landmarks.landmark))
print(len(results.face_landmarks.landmark))
print(len(results.right_hand_landmarks.landmark))
print(len(results.left_hand_landmarks.landmark))

33
468
21
21


In [30]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [31]:
with open('../data/csv/img_coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [23]:
len(list(np.array([[0, 0, 0, 0] * 33]).flatten()))


132

In [32]:
def coord_to_csv(imgs_folder, csv_path):
    counter = 0
    dir_list = os.listdir(imgs_folder)
    for folder in dir_list:
        class_name = folder
        imgs_list = os.listdir(imgs_folder + folder)
        print(f"Annotating {class_name}")
        for img in imgs_list:
            path = imgs_folder + folder + "/" + img
            with mp_holistic.Holistic(
            static_image_mode=True) as holistic:
                image = cv2.imread(path)
                image_height, image_width, _ = image.shape
                results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                # Extract Pose landmarks
                try:
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                except:
                    pose_row = list(np.array([[0, 0, 0, 0] * 33]).flatten())
                # Extract Face landmarks
                try:
                    face = results.face_landmarks.landmark
                    face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
                except:
                    face_row = list(np.array([[0, 0, 0, 0] * 468]).flatten())
                # extract left hand landmarks
                try:
                    left_hand = results.left_hand_landmarks.landmark
                    left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]\
                                                   for landmark in left_hand]).flatten())
                except:
                    left_hand_row = list(np.array([[0, 0, 0, 0] * 21]).flatten())
                # extract right hand landmarks
                try:
                    right_hand = results.left_hand_landmarks.landmark
                    right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]\
                                                   for landmark in right_hand]).flatten())
                except:
                    right_hand_row = list(np.array([[0, 0, 0, 0] * 21]).flatten())
                # Concate rows
                row = pose_row+face_row+left_hand_row+right_hand_row

                # Append class name 
                row.insert(0, class_name)

                # Export to CSV
                with open(csv_path, mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row)
            
                counter += 1
                if counter % 100 == 0:
                    print(f'{counter} imgs annotated')

In [33]:
coord_to_csv("../../../distracted_driver_dataset/imgs/train/", "../data/csv/img_coords.csv")

Annotating c0_safe
100 imgs annotated
200 imgs annotated
300 imgs annotated
400 imgs annotated
500 imgs annotated
600 imgs annotated
700 imgs annotated
800 imgs annotated
900 imgs annotated
1000 imgs annotated
1100 imgs annotated
1200 imgs annotated
1300 imgs annotated
1400 imgs annotated
1500 imgs annotated
1600 imgs annotated
1700 imgs annotated
1800 imgs annotated
1900 imgs annotated
2000 imgs annotated
2100 imgs annotated
2200 imgs annotated
2300 imgs annotated
2400 imgs annotated
Annotating c1_texting_right
2500 imgs annotated
2600 imgs annotated
2700 imgs annotated
2800 imgs annotated
2900 imgs annotated
3000 imgs annotated
3100 imgs annotated
3200 imgs annotated
3300 imgs annotated
3400 imgs annotated
3500 imgs annotated
3600 imgs annotated
3700 imgs annotated
3800 imgs annotated
3900 imgs annotated
4000 imgs annotated
4100 imgs annotated
4200 imgs annotated
4300 imgs annotated
4400 imgs annotated
4500 imgs annotated
4600 imgs annotated
4700 imgs annotated
Annotating c2_phonecal

In [34]:
pd.read_csv("../data/csv/img_coords.csv")

class        x1        y1        z1   v1        x2        y2  \
0         c0_safe  0.545086  0.168519 -0.356953  1.0  0.534952  0.129982   
1         c0_safe  0.451869  0.340804 -0.414619  1.0  0.433471  0.310055   
2         c0_safe  0.424279  0.346203 -0.320999  1.0  0.402844  0.316319   
3         c0_safe  0.335687  0.278907 -0.669487  1.0  0.320887  0.222015   
4         c0_safe  0.427172  0.244115 -0.251969  1.0  0.408947  0.205869   
...           ...       ...       ...       ...  ...       ...       ...   
15871  c6_talking  0.499460  0.202310 -0.494844  1.0  0.472200  0.166520   
15872  c6_talking  0.452590  0.290347 -0.327590  1.0  0.437952  0.255880   
15873  c6_talking  0.332026  0.424834 -0.587738  1.0  0.328176  0.371459   
15874  c6_talking  0.305156  0.401048 -0.553958  1.0  0.286867  0.356472   
15875  c6_talking  0.447381  0.274608 -0.621958  1.0  0.443139  0.221287   

             z2   v2        x3  ...      z541  v541      x542      y542  \
0     -0.338494  1.0  0.541106  ...  0.000000   0.0  0.000000  0.000000   
1     -0.396678  1.0  0.435723  ... -0.010021   0.0  0.927008  0.382200   
2     -0.321175  1.0  0.399517  ...  0.000000   0.0  0.000000  0.000000   
3     -0.638290  1.0  0.334692  ... -0.053340   0.0  0.787920  0.217617   
4     -0.227807  1.0  0.414374  ...  0.000705   0.0  1.004587  0.404081   
...         ...  ...       ...  ...       ...   ...       ...       ...   
15871 -0.467975  1.0  0.476481  ...  0.000000   0.0  0.000000  0.000000   
15872 -0.280452  1.0  0.445593  ...  0.000000   0.0  0.000000  0.000000   
15873 -0.563482  1.0  0.342044  ... -0.004542   0.0  0.820223  0.206265   
15874 -0.533201  1.0  0.296299  ...  0.015730   0.0  0.823949  0.210585   
15875 -0.597304  1.0  0.455274  ...  0.000000   0.0  0.000000  0.000000   

           z542  v542      x543      y543      z543  v543  
0      0.000000   0.0  0.000000  0.000000  0.000000   0.0  
1     -0.007705   0.0  0.937594  0.383414 -0.003735   0.0  
2      0.000000   0.0  0.000000  0.000000  0.000000   0.0  
3     -0.065578   0.0  0.792619  0.214140 -0.072630   0.0  
4      0.019041   0.0  0.994604  0.404329  0.023989   0.0  
...         ...   ...       ...       ...       ...   ...  
15871  0.000000   0.0  0.000000  0.000000  0.000000   0.0  
15872  0.000000   0.0  0.000000  0.000000  0.000000   0.0  
15873 -0.004373   0.0  0.833012  0.205601 -0.002697   0.0  
15874  0.016914   0.0  0.817067  0.213491  0.015468   0.0  
15875  0.000000   0.0  0.000000  0.000000  0.000000   0.0  

[15876 rows x 2173 columns]